TODO
* get all products
* download one each
* get the resoltion for each

In [1]:
import nesdis_gml_synergy.satlab as ngs

In [4]:
import nesdis_aws.nesdis_aws as ngsaws

In [15]:
import s3fs

In [57]:
import datetime

In [48]:
ngsaws.readme()

follow link for readme: https://docs.opendata.aws/noaa-goes16/cics-readme.html


In [41]:
products = ngsaws.available_products()

goes 16 and 17 products are identical


In [42]:
satellite = 16

In [49]:
prodser = products[satellite]

In [53]:
products = [p for p in prodser if p[-1] ==  'C']


In [168]:
reload(ngsaws)

<module 'nesdis_aws.nesdis_aws' from '/mnt/telg/prog/nesdis_aws/nesdis_aws/nesdis_aws.py'>

In [182]:
start = pd.to_datetime((datetime.datetime.now() - datetime.timedelta(3)).date()) + datetime.timedelta(hours=13)

In [183]:
info=[]
failure = []
for prod in products:
#     break
    try:

        que = ngsaws.AwsQuery(
                                path2folder_local='/mnt/telg/tmp/aws_tmp/',
                                satellite=satellite,
                                product=prod,
                                scan_sector='C',
                                start=start,
                                end=start + datetime.timedelta(hours = 3),#(days = 1),
                                no_of_days=None,
                                last_x_days=None,
                                max_no_of_files=100,
                            )

        temporal_resolution = (que.workplan.index[1:] - que.workplan.index[:-1]).mean()

        que.workplan = que.workplan.iloc[[1]]
        info.append({'product': prod, 
                     'workplan': que.workplan.copy(),
                     'temp_res': temporal_resolution,
                     'available_since': que.product_available_since}) 
        que.download()
    except:
        failure.append(prod)
        print(f"{prod} failed")
        

2000
2017
2019
2000
2017
2000
2017
2019
2000
2017
2000
2017
2000
2017
2019
2019
2019
2021
2019
2019
2017
2000
2017
2019
2019
2000
2017
2019
2019


In [196]:
import atmPy.data_archives.NOAA_ESRL_GMD_GRAD.surfrad.surfrad as surfrad

In [198]:
type(surfrad.network.stations.Table_Mountain)

atmPy.general.measurement_site.Station

In [197]:
surfrad.network.stations.Table_Mountain.lat, surfrad.network.stations.Table_Mountain.lon

(40.12498, -105.2368)

In [199]:
site = surfrad.network.stations.Table_Mountain

In [202]:
hasattr(site, 'lat') and hasattr(site, 'lon')

True

In [ ]:
reload(ngs)

In [186]:
info_df = pd.DataFrame(info.copy())

In [242]:
info_df['lon_res'] = np.nan
info_df['lat_res'] = np.nan
info_df['spatial_resolution'] = np.nan
info_df['title'] = np.nan
info_df['summary'] = np.nan

In [264]:
for idx, row in info_df.iterrows():
    if not np.isnan(row.lon_res):
        continue
    
    satinst  = ngs.open_file(row.workplan.path2file_local.iloc[0])
    try:
        res = satinst.get_resolution()

        info_df.loc[idx, 'lon_res'] = res.loc[0,'lon']
        info_df.loc[idx, 'lat_res'] = res.loc[0,'lat']
    except:
        pass
    info_df.loc[idx, 'spatial_resolution'] = satinst.ds.attrs['spatial_resolution']
    info_df.loc[idx, 'title'] = satinst.ds.attrs['title']
    info_df.loc[idx, 'summary'] = satinst.ds.attrs['summary']
#     except:
#         continue

In [266]:
info_df_bk = info_df.copy()

In [282]:
dft = info_df.drop(['workplan'], axis=1)

In [283]:
dft['temp_res_min'] = dft.apply(lambda row: row.temp_res/pd.to_timedelta(1,'min'), axis = 1)

In [284]:
dft = dft.drop(['temp_res'], axis=1)

In [285]:
dft

,product,available_since,lon_res,lat_res,spatial_resolution,title,summary,temp_res_min
0,ABI-L1b-RadC,2017-03-01,3.452556,2.673190,2km at nadir,ABI L1b Radiances,Single reflective band ABI L1b Radiance Produc...,0.304348
1,ABI-L2-ACHAC,2019-12-03,17.281100,13.363345,10km at nadir,ABI L2 Cloud Top Height,The Cloud Top Height product consists of the h...,5.000000
2,ABI-L2-ACMC,2017-04-20,3.452556,2.673190,2km at nadir,ABI L2 Clear Sky Mask,The Clear Sky Mask product consists of a binar...,5.000000
3,ABI-L2-ACTPC,2017-05-17,3.452556,2.673190,2km at nadir,ABI L2 Cloud Top Phase,The Cloud Top Phase product consists of cloud ...,5.000000
4,ABI-L2-ADPC,2019-12-04,3.452556,2.673190,2km at nadir,ABI L2 Aerosol Detection,The Aerosol Detection product consists of thre...,10.000000
5,ABI-L2-AODC,2017-05-25,3.452556,2.673190,2km at nadir,ABI L2 Aerosol Optical Depth,The Aerosol Optical Depth at 550 nm product co...,5.000000
6,ABI-L2-CMIPC,2017-03-01,3.452556,2.673190,2km at nadir,ABI L2 Cloud and Moisture Imagery,Single reflective band Cloud and Moisture Imag...,0.304348
7,ABI-L2-CODC,2017-06-09,3.452556,2.673190,2km at nadir,ABI L2 Cloud Optical Depth at 640 nm,The Cloud Optical Depth product consists of pi...,5.000000
8,ABI-L2-CPSC,2019-12-06,3.452556,2.673190,2km at nadir,ABI L2 Cloud Particle Size,The Cloud Particle Size product consists of pi...,5.000000
9,ABI-L2-CTPC,2019-12-06,17.281100,13.363345,10km at nadir,ABI L2 Cloud Top Pressure,The Cloud Top Pressure product consists of the...,5.000000


In [286]:
dft.to_csv('product_info.csv')